In [234]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, date_format, concat_ws, to_timestamp, to_date, expr, StringType, collect_list, udf, join
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("FB").getOrCreate()

df = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0.csv", header=True, inferSchema=True)
df2 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (1).csv", header=True, inferSchema=True)
df3 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (2).csv", header=True, inferSchema=True)
df4 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (3).csv", header=True, inferSchema=True)


ImportError: cannot import name 'join' from 'pyspark.sql.functions' (c:\Python312\Lib\site-packages\pyspark\sql\functions.py)

In [ ]:
df.show(5)

+---+----------+-------------------+-----------+-------------+----+----+---+----+----+---+---------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+----+----+-----+----+---+----+----+----+----+----+----+----+----+----+-----+--------+--------+-----+-----+-------+-------+-------+-------+-----+-------+-------+----+----+------+------+------+------+------+------+------+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+---------+---------+------+------+--------+--------+--------+--------+-----+--------+--------+-----+-----+-------+-------+-------+-------+
|Div|      Date|               Time|   HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|  Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA| PSH| PSD|  PSA| WHH|WHD| WHA| VCH| VCD| VCA|MaxH|MaxD|MaxA|AvgH|AvgD| AvgA|B365>2.5|B365<2.5|P>2.5|P<2.5|Max>2.5|Max<2.5|Avg>2.5|Avg<2.5|

In [ ]:
df = df.union(df2)
df = df.union(df3)
df = df.union(df4)

In [ ]:
df = df.drop("Div")
df = df.select("Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR")

In [ ]:
dfHome = df.select(col("Time"), col("HomeTeam").alias("Team"), col("FTR"))
dfAway = df.select(col("Time"), col("AwayTeam").alias("Team"), col("FTR"))

dfForm = dfHome.union(dfAway).orderBy("Time")
dfForm = dfForm.select("Time", "Team", "FTR")

#Last 5 game results for the home team
window_spec = Window.partitionBy("Team").orderBy("Time").rowsBetween(Window.unboundedPreceding, Window.currentRow)

dfForm = dfForm.withColumn("results_list", collect_list("FTR").over(window_spec))

dfForm.createOrReplaceTempView("games")
query = """
SELECT *,
    CASE 
        WHEN size(results_list) >= 5 THEN concat_ws('', slice(results_list, size(results_list) - 4, 5))
        ELSE concat_ws('', results_list)
    END AS TeamForm
FROM games
"""

dfForm = spark.sql(query)
dfForm = dfForm.drop("results_list")

df = df.join(dfForm.withColumnRenamed("Team", "HomeTeam").withColumnRenamed("TeamForm", "HomeTeamForm"),
             on=[df.HomeTeam == dfForm.Team, df.Time == dfForm.Time], how="left")

df = df.join(dfForm.withColumnRenamed("Team", "AwayTeam").withColumnRenamed("TeamForm", "AwayTeamForm"),
             on=[df.AwayTeam == dfForm.Team, df.Time == dfForm.Time], how="left")

dfForm.show()

AnalysisException: [MISSING_ATTRIBUTES.RESOLVED_ATTRIBUTE_MISSING_FROM_INPUT] Resolved attribute(s) "Team" missing from "Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "Time", "HomeTeam", "FTR", "HomeTeamForm" in operator !Join RightOuter, ((HomeTeam#89101 = Team#90975) AND (Time#91441 = Time#91441)). ;
!Join RightOuter, ((HomeTeam#89101 = Team#90975) AND (Time#91441 = Time#91441))
:- Project [Date#89099, Time#89100, HomeTeam#89101, AwayTeam#89102, FTHG#89103, FTAG#89104, FTR#89105, HTHG#89106, HTAG#89107, HTR#89108, Referee#89109, HS#89110, AS#89111, HST#89112, AST#89113, HF#89114, AF#89115, HC#89116, AC#89117, HY#89118, AY#89119, HR#89120, AR#89121]
:  +- Project [Date#89099, Time#89100, HomeTeam#89101, AwayTeam#89102, FTHG#89103, FTAG#89104, FTR#89105, HTHG#89106, HTAG#89107, HTR#89108, Referee#89109, HS#89110, AS#89111, HST#89112, AST#89113, HF#89114, AF#89115,

In [ ]:
df.groupBy("Referee").count().orderBy("count", ascending=False).show(300)

+------------+-----+
|     Referee|count|
+------------+-----+
|    A Taylor|  113|
|    M Oliver|  108|
|   P Tierney|  103|
|    C Pawson|   89|
|   S Attwell|   85|
|    S Hooper|   81|
|    A Madley|   79|
|     D Coote|   78|
|  C Kavanagh|   73|
|     R Jones|   69|
|    P Bankes|   59|
|  A Marriner|   57|
|   D England|   56|
|  M Atkinson|   52|
|      M Dean|   51|
|      J Moss|   49|
|   J Gillett|   46|
|    J Brooks|   45|
|    K Friend|   43|
|     G Scott|   37|
| M Salisbury|   32|
|  T Robinson|   23|
|   T Bramall|   19|
|T Harrington|   19|
|   S Barrott|   15|
|     L Mason|   11|
|      D Bond|    9|
|     J Smith|    5|
|   S Allison|    3|
|    R Madley|    2|
|     R Welch|    2|
|     L Smith|    2|
|   M Donohue|    2|
|  J Gillett |    1|
|     S Singh|    1|
|      A Moss|    1|
+------------+-----+



In [ ]:
df = df.withColumn("corrected_date", expr("concat('20', substr(date, 9, 2), '-', substr(date, 4, 2), '-', substr(date, 1, 2))"))
df = df.withColumn("corrected_date_type", to_date(col("corrected_date")))
df = df.withColumn("updated_datetime", (to_timestamp(concat_ws(' ', date_format(col("corrected_date_type"), 'yyyy-MM-dd'), date_format(col("Time"), 'HH:mm:ss')))))
df = df.drop("Date", "Time", "corrected_date")
df = df.withColumnRenamed("corrected_date_type", "Date")
df = df.withColumnRenamed("updated_datetime", "Time")


In [ ]:
df.head(5)

[Row(HomeTeam='Burnley', AwayTeam='Man City', FTHG=0, FTAG=3, FTR='A', HTHG=0, HTAG=2, HTR='A', Referee='C Pawson', HS=6, AS=17, HST=1, AST=8, HF=11, AF=8, HC=6, AC=5, HY=0, AY=0, HR=1, AR=0, Date=datetime.date(2023, 8, 11), Time=datetime.datetime(2023, 8, 11, 20, 0)),
 Row(HomeTeam='Arsenal', AwayTeam="Nott'm Forest", FTHG=2, FTAG=1, FTR='H', HTHG=2, HTAG=0, HTR='H', Referee='M Oliver', HS=15, AS=6, HST=7, AST=2, HF=12, AF=12, HC=8, AC=3, HY=2, AY=2, HR=0, AR=0, Date=datetime.date(2023, 8, 12), Time=datetime.datetime(2023, 8, 12, 12, 30)),
 Row(HomeTeam='Bournemouth', AwayTeam='West Ham', FTHG=1, FTAG=1, FTR='D', HTHG=0, HTAG=0, HTR='D', Referee='P Bankes', HS=14, AS=16, HST=5, AST=3, HF=9, AF=14, HC=10, AC=4, HY=1, AY=4, HR=0, AR=0, Date=datetime.date(2023, 8, 12), Time=datetime.datetime(2023, 8, 12, 15, 0)),
 Row(HomeTeam='Brighton', AwayTeam='Luton', FTHG=4, FTAG=1, FTR='H', HTHG=1, HTAG=0, HTR='H', Referee='D Coote', HS=27, AS=9, HST=12, AST=3, HF=11, AF=12, HC=6, AC=7, HY=2, AY=2